In [ ]:
//程式碼由 Boris @ http://www.youtube.com/borispcp 設計，歡迎使用但請保留此聲明。

var form = FormApp.getActiveForm();
var responseSpreadsheetID = form.getDestinationId();    //取得存放表單回應的試算表的ID
var lineNotifyTokens;    //存放 Line Notify 權杖
var telegramBotToken;    //存放 Telegram Bot 權杖
var telegramReceivedIDs; //存放接收資料的使用者（或群組）ID
var webAppURL;           //存放此程式部署為網路應用程式後的網址

//移除試算表空儲存格
function transformArray(arrayData) {
  var returnArray = [];
  for (var i = 0; i < arrayData.length; i++) {
    if (arrayData[i] !== "") {returnArray.push(arrayData[i].toString().replace(/\r?\n|\r/g,""));}
  }
  return returnArray;
}

//取得使用者回應並建立 Line Notify 及 Telegram 訊息
function getFormData(e) {
  getConfigData();
  var currentItemResponses = e.response.getItemResponses();
  var message = "\n\n" + form.getTitle() + "\n\n";    //存放訊息的變數，並將表單標題放入變數內

  //用 for 迴圈取得所有有作答的題目及使用者回應並加入 message 變數內。
  // currentItemResponses[i].getItem().getTitle() 是題目，currentItemResponses[i].getResponse() 是使用者回應
  for(var i = 0; i < currentItemResponses.length; i++) {
    if (currentItemResponses[i].getResponse() !== "") {message += currentItemResponses[i].getItem().getTitle() + "：" + currentItemResponses[i].getResponse() + "\n\n";}
  }

  //將表單送出時間加入 message 變數內
  message += "通報時間：" + new Date(e.response.getTimestamp()).toLocaleString('zh-TW', {timeZone: 'Asia/Taipei'});

  var sendMessageData = [];
  //製作 Line Notify 訊息
  if (lineNotifyTokens.length !== 0) {
    sendMessageData = sendMessageData.concat(createLineNotifyMessage(message, lineNotifyTokens));
  }

  //製作 Telegram 訊息
  if (telegramReceivedIDs.length !== 0 && telegramBotToken !== "") {
    sendMessageData = sendMessageData.concat(createTelegramBotMessage(message, telegramReceivedIDs));
  }
  if (sendMessageData.length > 0) {UrlFetchApp.fetchAll(sendMessageData);}
}

//製作 Line Notify 訊息
function createLineNotifyMessage(message, tokens){
  var request = [];
  for (var i = 0; i < tokens.length; i++) {
    var options =
    {
      "url" : "https://notify-api.line.me/api/notify",
      "muteHttpExceptions" : true,
      "method"  : "post",
      "payload" : {"message" : message},
      "headers" : {"Authorization" : "Bearer " + tokens[i]}
    };
    request.push(options);
  }
  return request;
}

//製作 Telegram 訊息
function createTelegramBotMessage(message, clientIDs) {
  var request = [];
  for (var i = 0; i < clientIDs.length; i++) {
    var payload = {
      "chat_id": clientIDs[i],
      "text": message,
    };
    var options = {
      "url" : "https://api.telegram.org/bot" + telegramBotToken + "/" + "sendMessage",
      "muteHttpExceptions" : true,
      "method": "post",
      "contentType": "application/json",
      "payload": JSON.stringify(payload)
    };
    request.push(options);
  }
  return request;
}

//設定 Telegram Bot 的 Webhook
function telegramSetWebhook() {
  getConfigData();
  var ui = FormApp.getUi();
  if (telegramBotToken === ""){ui.alert("尚未輸入 Telegram Bot Token，請至試算表的「參數設定」工作表進行設定"); return;}
  if (ScriptApp.getService().getUrl() === null){ui.alert("程式尚未部署為網路應用程式，請至「指令碼編輯器內進行部屬」"); return;}
  if (webAppURL === ""){ui.alert("尚未輸入GAS網頁應用程式網址，請至試算表的「參數設定」工作表進行設定"); return;}
  console.log(JSON.parse(UrlFetchApp.fetch("https://api.telegram.org/bot" + telegramBotToken + "/" + "deleteWebhook")));    //刪除舊的 Telegram Bot Webhook 資訊
  try {
    var telegramSetResult = JSON.parse(UrlFetchApp.fetch("https://api.telegram.org/bot" + telegramBotToken + "/" + "setWebhook?url=" + webAppURL));    //設定新的 Telegram Bot Webhook 資訊
    console.log(telegramSetResult);
    if (telegramSetResult.ok == true){
      ui.alert("Telegram Bot Webhook 設定完成");
    }
    else {
      ui.alert("Telegram Bot Webhook 設定失敗，請至指令碼編輯器參閱「執行項目」的錯誤訊息");
    }
  }
  catch(Error) {
    console.error(Error);
    ui.alert("Telegram Bot Webhook 設定失敗，請至指令碼編輯器參閱「執行項目」的錯誤訊息");
  }
}

//查詢接收者 Telegram ID
function doPost(e) {
  getConfigData();
  var userData = JSON.parse(e.postData.contents);    //取得 Telegram Bot 收到的資訊
  var clientID = userData.message.chat.id;    //取出此訊息的 Chat ID
  if (!userData.message.text) {return;}    //不對文字以外的訊息進行處理

  //如果使用者輸入了 FindMyID 關鍵字（不論大小寫），則回應 Chat ID 給使用者
  if (userData.message.text.toLowerCase() == "findmyid") {
    var replyMessage = "您接收資料用的 ID 是「" + clientID + "」，請將此 ID 告知此官方帳號的擁有者進行設定。";
    UrlFetchApp.fetchAll(createTelegramBotMessage(replyMessage,　[clientID]));
  }
}

//取得參數設定檔資料
function getConfigData() {
  var spreadSheetConfig = SpreadsheetApp.openById(responseSpreadsheetID);    //連接存放表單回應的試算表
  var sheetConfig = spreadSheetConfig.getSheetByName("參數設定");    //連接「參數設定」工作表
  var sheetConfigData = sheetConfig.getSheetValues(1, 2, sheetConfig.getLastRow(), sheetConfig.getLastColumn() - 1);    //取得「參數設定」工作表的資料
  lineNotifyTokens = transformArray(sheetConfigData[0]);    //取得「參數設定」工作表內所有的 Line Notify 權杖
  telegramBotToken = sheetConfigData[1][0].replace(/\r?\n|\r/g,"");                 //取得「參數設定」工作表內所記錄的 Telegram Bot 權杖
  telegramReceivedIDs = transformArray(sheetConfigData[2]); //取得「參數設定」工作表內所有接收資料的 Telegram 使用者（或聊天室）ID
  webAppURL = sheetConfigData[3][0].replace(/\r?\n|\r/g,"");  //取得「參數設定」工作表內所記錄的此網路應用程式的網址
}

//建立觸發條件
function createTrigger() {
  var allTriggers = ScriptApp.getProjectTriggers();

  var checkTemp = allTriggers.filter(function(item, index, array){
    return item.getHandlerFunction() == "getFormData";
  });
  if (checkTemp.length == 0) {
    ScriptApp.newTrigger("getFormData")
             .forForm(form.getId())
             .onFormSubmit()
             .create();
  }
  var ui = FormApp.getUi();
  ui.alert("觸發條件已建立完成");
}

//在表單裡建立選單供使用者點選
function onOpen() {
  var ui = FormApp.getUi();
  ui.createMenu("將 Google 表單的使用者回應傳送給 Line 及 Telegram 2021 專用選單")
      .addItem("建立觸發條件", "createTrigger")
      .addSeparator()
      .addItem("設定 Telegram Bot Webhook", "telegramSetWebhook")
      .addToUi();
}